# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [106]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-eff9677b66-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

## Task 1
Who invented the theory of relativity? (return the IRI and the label of the inventor).

In [107]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q43514 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('

In [108]:
# Check the property using P61, Who discoverer or  who is the Inventor
# write your final query
# query example
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q43514 wdt:P61 ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


Final query for this task

## Task 2
Identify the BGP for physicist

In [110]:
# write your queries


queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q169470 ?p ?object .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 's

In [111]:
# write your queries
# Try to retrive data using Alber Einstein point which is Q937

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937 ?p ?object .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2861'), ('name', 'Leidse Hoogleraren ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2930'), ('name', 'INSPIRE-HEP author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6231'), ('name', 'BDELIS ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9044'), ('name', 'GEPRIS-Historisch ID (Person)')]
[('p', 'http://www.wikidata.org/prop/direct/P4116'), ('name', 'JewAge person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'ht

In [112]:
# write your queries
# Try to retrive data using the point which is P106 for the occupation to check if they are physicist

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('object', 'http://www.wikidata.org/entity/Q16389557'), ('name', 'philosopher of science')]
[('object', 'http://www.wikidata.org/entity/Q121594'), ('name', 'professor')]
[('object', 'http://www.wikidata.org/entity/Q1231865'), ('name', 'pedagogue')]
[('object', 'http://www.wikidata.org/entity/Q15980158'), ('name', 'non-fiction writer')]
[('object', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher')]
[('object', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
[('object', 'http://www.wikidata.org/entity/Q170790'), ('name', 'mathematician')]
[('object', 'http://www.wikidata.org/entity/Q205375'), ('name', 'inventor')]
[('object', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('object', 'http://www.wikidata.org/entity/Q37226'), ('name', 'teacher')]
[('object', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
[('obj

In [113]:
#Q937 is a point for Albert Einstein we are using this because This person was physicist
# write your final query
# write your queries
# Try to retrive data using the point which is P106 for the occupation to check if they are physicist

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q937 wdt:P106 ?object .
   # get the label
   ?object sc:name ?name.
   FILTER REGEX(?name, "physicist.*")
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('object', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
2


Final query for this task

In [115]:
# Retrived BGP data of Physicist
# write your final query
# write your queries
# Try to retrive data using the point which is P106 for the occupation to check if they are physicist

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q169470 ?object ?o.
   # get the label
   ?object sc:name ?name.
   #FILTER REGEX(?name, "citizen")
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('object', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('object', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('object', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('object', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('object', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('object', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('object', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('object', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('object', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('objec

## Task 3
Identify the BGP for Nobel Prize

In [48]:
# write your queries
# p166 THIS IS FOR AWARD CHECK ABOVE THIS using Albert Einstein link which is Q937

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q937 wdt:P166 ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1522873'), ('name', 'Josiah Willard Gibbs Lectureship')]
[('object', 'http://www.wikidata.org/entity/Q317038'), ('name', 'Max Planck Medal')]
[('object', 'http://www.wikidata.org/entity/Q1419938'), ('name', 'Matteucci Medal')]
[('object', 'http://www.wikidata.org/entity/Q14906020'), ('name', 'Foreign Member of the Royal Society')]
[('object', 'http://www.wikidata.org/entity/Q15056034'), ('name', 'Pour le Mérite for Sciences and Arts')]
[('object', 'http://www.wikidata.org/entity/Q25287226'), ('name', 'Honorary doctor of the University of Geneva')]
[('object', 'http://www.wikidata.org/entity/Q28003'), ('name', 'Copley Medal')]
[('object', 'http://www.wikidata.org/entity/Q28861731'), ('name', 'honorary doctor of the Hebrew University of Jerusalem')]
[('object', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('object', 'http://www.wikidata.org/entity/Q42309226'), ('name', 'honorary doctor of the Princeton

In [52]:
# NOw we get the Nobel Prize in Physics on Q38104

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38104 ?p ?object .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3553')

In [54]:
# NOw we get the Nobel Prize in Physics on Q38104 using instances of P31

# we just get Physics award using this P31 instances

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P31 ?object .
   # get the label
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('object', 'http://www.wikidata.org/entity/Q107494520'), ('name', 'physics award')]
1


Final query for this task

In [62]:
# write your final query

# NOw we get the Nobel Prize in Physics on Q38104 using instances of P31
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P279 ?object .
   # get the label 
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('object', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


In [117]:

# write your final query
# write your queries
# Retrived BGP data of Nobel Prize

queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q38104 ?object ?o.
   # get the label
   ?object sc:name ?name.
   #FILTER REGEX(?name, "Nobel Prize")
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('object', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('object', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('object', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('object', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('object', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('object', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('object', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('object', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('object', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('object', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('object', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora top

## Task 4
Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

In [118]:
# write your queries
#wd:Q937 P166

#Gettign the names of all Physicists and their country for citizenship

queryString = """
SELECT DISTINCT ?object ?name ?country ?cname
WHERE {
   # bind something
   ?object ?o wd:Q169470;
   wdt:P27 ?country.
   #wdt:P166 wd:Q7191.
   # get the label
   ?object sc:name ?name.
   ?country sc:name ?cname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('object', 'http://www.wikidata.org/entity/Q680'), ('name', 'Alessandro Volta'), ('country', 'http://www.wikidata.org/entity/Q209857'), ('cname', 'Kingdom of Lombardy–Venetia')]
[('object', 'http://www.wikidata.org/entity/Q310757'), ('name', 'Ruđer Josip Bošković'), ('country', 'http://www.wikidata.org/entity/Q208169'), ('cname', 'Republic of Ragusa')]
[('object', 'http://www.wikidata.org/entity/Q65183'), ('name', 'Johann Gottlieb Friedrich von Bohnenberger'), ('country', 'http://www.wikidata.org/entity/Q159631'), ('cname', 'Kingdom of Württemberg')]
[('object', 'http://www.wikidata.org/entity/Q57171'), ('name', 'Julius von Mayer'), ('country', 'http://www.wikidata.org/entity/Q159631'), ('cname', 'Kingdom of Württemberg')]
[('object', 'http://www.wikidata.org/entity/Q62981'), ('name', 'Johann Georg Christian Lehmann'), ('country', 'http://www.wikidata.org/entity/Q704288'), ('cname', 'Duchy of Holstein')]
[('object', 'http://www.wikidata.org/entity/Q9036'), ('name', 'Nikola Tes

In [119]:
# write your queries
#wd:Q937 P166


#('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
queryString = """
SELECT DISTINCT ?objec ?name ?country ?cname
WHERE {
   # bind something
   wd:Q142 ?objec ?o.
   # get the label
   ?objec sc:name ?name.
   #?country sc:name ?cname.
   FILTER(REGEX(?name,"continent"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('objec', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
1


In [120]:
# write your queries
#wd:Q937 P166


#('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
queryString = """
SELECT DISTINCT ?objec ?name ?country ?cname
WHERE {
   # bind something
   wd:Q142 wdt:P30 ?objec.
   # get the label
   ?objec sc:name ?name.
   #?country sc:name ?cname.
   #FILTER(REGEX(?name,"continent"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('objec', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
1


In [121]:
# write your queries
#wd:Q937 P166
#Filtering data for Europe

queryString = """
SELECT DISTINCT ?object ?name ?country ?cname ?aw
WHERE {
   # bind something
   ?object ?o wd:Q169470;
   wdt:P166 wd:Q38104;
   wdt:P27 ?country.
   ?country wdt:P30 wd:Q46.
   #.
   # get the label
   ?object sc:name ?name.
   ?country sc:name ?cname.
   #?award sc:name ?aw.

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('object', 'http://www.wikidata.org/entity/Q76766'), ('name', 'Gerd Binnig'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('object', 'http://www.wikidata.org/entity/Q57072'), ('name', 'Otto Stern'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('object', 'http://www.wikidata.org/entity/Q71024'), ('name', 'Klaus von Klitzing'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('object', 'http://www.wikidata.org/entity/Q65807'), ('name', 'Reinhard Genzel'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('object', 'http://www.wikidata.org/entity/Q71022'), ('name', 'Ernst Ruska'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('object', 'http://www.wikidata.org/entity/Q71023'), ('name', 'Alfred Kastler'), ('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France')]
[('object', 'http://www.wikidata.org/entity/Q71023'), ('name',

Final query for this task

In [105]:
# write your final query
# write your queries
#wd:Q937 P166
# write your final query
# FInal Result for Awards

queryString = """
SELECT DISTINCT (COUNT(?object) as ?NumOfAwards) ?country ?cname
WHERE {
   # bind something
   ?object ?o wd:Q169470;
   wdt:P166 wd:Q38104;
   wdt:P27 ?country.
   ?country wdt:P30 wd:Q46.
   
   # get the label
   ?object sc:name ?name.
   ?country sc:name ?cname.

}
GROUP BY ?country ?cname
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('NumOfAwards', '1'), ('country', 'http://www.wikidata.org/entity/Q171150'), ('cname', 'Kingdom of Hungary')]
[('NumOfAwards', '2'), ('country', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary')]
[('NumOfAwards', '1'), ('country', 'http://www.wikidata.org/entity/Q215'), ('cname', 'Slovenia')]
[('NumOfAwards', '4'), ('country', 'http://www.wikidata.org/entity/Q38'), ('cname', 'Italy')]
[('NumOfAwards', '25'), ('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('NumOfAwards', '4'), ('country', 'http://www.wikidata.org/entity/Q41304'), ('cname', 'Weimar Republic')]
[('NumOfAwards', '4'), ('country', 'http://www.wikidata.org/entity/Q43287'), ('cname', 'German Empire')]
[('NumOfAwards', '6'), ('country', 'http://www.wikidata.org/entity/Q174193'), ('cname', 'United Kingdom of Great Britain and Ireland')]
[('NumOfAwards', '1'), ('country', 'http://www.wikidata.org/entity/Q533534'), ('cname', 'Cisleithania')]
[('NumOfAwards', '4'), ('country', 'http:/

## Task 5
Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label).

In [130]:
# write your queries

 
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   # bind something
   ?s wdt:P166 ?nPrize;
   wdt:P19 ?place.
   ?country wdt:P31 wd:Q6256.
   ?place wdt:P17 ?country.
   # get the label
   ?s sc:name ?Sname.
   ?country sc:name ?countryName.
 
}
GROUP BY ?country ?countryName
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('country', 'http://www.wikidata.org/entity/Q754'), ('countryName', 'Trinidad and Tobago')]
[('country', 'http://www.wikidata.org/entity/Q854'), ('countryName', 'Sri Lanka')]
[('country', 'http://www.wikidata.org/entity/Q865'), ('countryName', 'Taiwan')]
[('country', 'http://www.wikidata.org/entity/Q921'), ('countryName', 'Brunei')]
[('country', 'http://www.wikidata.org/entity/Q929'), ('countryName', 'Central African Republic')]
[('country', 'http://www.wikidata.org/entity/Q953'), ('countryName', 'Zambia')]
[('country', 'http://www.wikidata.org/entity/Q962'), ('countryName', 'Benin')]
[('country', 'http://www.wikidata.org/entity/Q963'), ('countryName', 'Botswana')]
[('country', 'http://www.wikidata.org/entity/Q967'), ('countryName', 'Burundi')]
[('country', 'http://www.wikidata.org/entity/Q974'), ('countryName', 'Democratic Republic of the Congo')]
[('country', 'http://www.wikidata.org/entity/Q907112'), ('countryName', 'Transnistria')]
[('country', 'http://www.wikidata.org/ent

Final query for this task

In [135]:
# write your final query

# write your queries

 




## Task 6
Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

In [139]:
# write your queries

# We get the Nobel Prize in Physics on Q38104 using instances of P31
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
   wd:Q38104 wdt:P31 ?object .
   # get the label.
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q107494520'), ('name', 'physics award')]
1


In [142]:
# write your queries

# We get the Nobel Prize in Physics on Q38104 using instances of P31
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
    ?object wdt:P31 wd:Q107494520   .
   # get the label.
   ?object sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1168915'), ('name', 'Swan Medal')]
[('object', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('object', 'http://www.wikidata.org/entity/Q56610568'), ('name', 'Rutherford Memorial Medal in Physics')]
[('object', 'http://www.wikidata.org/entity/Q96402465'), ('name', 'Rolf Wideroe Prize')]
4


In [146]:
# write your queries

# We get the Nobel Prize in Physics on Q38104 using instances of P31
queryString = """
SELECT DISTINCT ?object ?name
WHERE {
   # bind something
     wd:Q38104 ?object ?name   .
   # get the label P31.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('object', 'http://schema.org/name'), ('name', 'Nobel Prize in Physics')]
1


In [184]:

# Retrive all the physicist who won the Nobel Prize and all the discoverer
queryString = """
SELECT DISTINCT ?s ?sName ?disc ?discoveries
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104.
   ?discoveries wdt:P61 ?s.
   
   # get the label
   ?s sc:name ?sName.
   ?discoveries sc:name ?name.
 
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q71024'), ('sName', 'Klaus von Klitzing'), ('discoveries', 'http://www.wikidata.org/entity/Q1047822')]
[('s', 'http://www.wikidata.org/entity/Q71022'), ('sName', 'Ernst Ruska'), ('discoveries', 'http://www.wikidata.org/entity/Q132560')]
[('s', 'http://www.wikidata.org/entity/Q191922'), ('sName', 'Frederick Reines'), ('discoveries', 'http://www.wikidata.org/entity/Q1066748')]
[('s', 'http://www.wikidata.org/entity/Q57068'), ('sName', 'Wilhelm Wien'), ('discoveries', 'http://www.wikidata.org/entity/Q214336')]
[('s', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('discoveries', 'http://www.wikidata.org/entity/Q83913')]
[('s', 'http://www.wikidata.org/entity/Q57077'), ('sName', 'Ferdinand Braun'), ('discoveries', 'http://www.wikidata.org/entity/Q174320')]
[('s', 'http://www.wikidata.org/entity/Q58978'), ('sName', 'Max Born'), ('discoveries', 'http://www.wikidata.org/entity/Q852824')]
[('s', 'http://www.wikidata.org/en

In [185]:

#Arrange Group by the inventor and Discovries


queryString = """
SELECT DISTINCT ?s ?scienctistName (COUNT(?object) as ?TotalNumerofDiscoveries)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104.
   ?object wdt:P61 ?s.
   
   # get the label
   ?object sc:name ?discName.
   ?s sc:name ?scienctistName.
   
 
}
GROUP BY ?s ?scienctistName
HAVING (COUNT(?object) >=3 )
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q193803'), ('scienctistName', 'Roger Penrose'), ('TotalNumerofDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q160518'), ('scienctistName', 'Gustaf Dalén'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q8753'), ('scienctistName', 'Enrico Fermi'), ('TotalNumerofDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q123975'), ('scienctistName', 'Michel Mayor'), ('TotalNumerofDiscoveries', '12')]
[('s', 'http://www.wikidata.org/entity/Q173028'), ('scienctistName', 'Emilio G. Segrè'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q124013'), ('scienctistName', 'Didier Queloz'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q937'), ('scienctistName', 'Albert Einstein'), ('TotalNumerofDiscoveries', '10')]
[('s', 'http://www.wikidata.org/entity/Q47480'), ('scienctistName', 'Paul Dirac'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.or

Final query for this task

In [186]:
# write your final query

#Arrange Group by the inventor and Discovries


queryString = """
SELECT DISTINCT ?s ?scienctistName (COUNT(?object) as ?TotalNumerofDiscoveries)
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104.
   ?object wdt:P61 ?s.
   
   # get the label
   ?object sc:name ?discName.
   ?s sc:name ?scienctistName.
   
 
}
GROUP BY ?s ?scienctistName
HAVING (COUNT(?object) >=3 )
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q193803'), ('scienctistName', 'Roger Penrose'), ('TotalNumerofDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q160518'), ('scienctistName', 'Gustaf Dalén'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q8753'), ('scienctistName', 'Enrico Fermi'), ('TotalNumerofDiscoveries', '6')]
[('s', 'http://www.wikidata.org/entity/Q123975'), ('scienctistName', 'Michel Mayor'), ('TotalNumerofDiscoveries', '12')]
[('s', 'http://www.wikidata.org/entity/Q173028'), ('scienctistName', 'Emilio G. Segrè'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q124013'), ('scienctistName', 'Didier Queloz'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.org/entity/Q937'), ('scienctistName', 'Albert Einstein'), ('TotalNumerofDiscoveries', '10')]
[('s', 'http://www.wikidata.org/entity/Q47480'), ('scienctistName', 'Paul Dirac'), ('TotalNumerofDiscoveries', '3')]
[('s', 'http://www.wikidata.or

## Task 7
Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [189]:
# write your queries



queryString = """
SELECT DISTINCT ?s ?scienctistName ?university ?universityName
WHERE {
   # bind something
   ?s wdt:P166 wd:Q38104;
    wdt:P106 wd:Q1622272;
    wdt:P108 ?university;
    wdt:P19 ?placeofBirth.
   
   # get the label
   ?s sc:name ?scienctistName.
   ?university sc:name ?universityName.
   
 
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q133747'), ('scienctistName', 'William Henry Bragg'), ('university', 'http://www.wikidata.org/entity/Q15574'), ('universityName', 'University of Adelaide')]
[('s', 'http://www.wikidata.org/entity/Q181363'), ('scienctistName', 'Brian David Josephson'), ('university', 'http://www.wikidata.org/entity/Q4380853'), ('universityName', 'Missouri University of Science and Technology')]
[('s', 'http://www.wikidata.org/entity/Q41269'), ('scienctistName', 'Henri Becquerel'), ('university', 'http://www.wikidata.org/entity/Q838691'), ('universityName', "Muséum national d'histoire naturelle")]
[('s', 'http://www.wikidata.org/entity/Q71013'), ('scienctistName', 'Theodor W. Hänsch'), ('university', 'http://www.wikidata.org/entity/Q829535'), ('universityName', 'Max Planck Institute of Quantum Optics')]
[('s', 'http://www.wikidata.org/entity/Q57100'), ('scienctistName', 'Maria Goeppert Mayer'), ('university', 'http://www.wikidata.org/entity/Q649120'), ('univ

In [191]:
# write your queries

queryString = """
SELECT DISTINCT ?p ?pName ?object ?objName
WHERE {
   # bind something
    wd:Q15574 ?p ?object.
    
   
   # get the label
   ?p sc:name ?pName.
   ?object sc:name ?objectName.
   
 
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P488'), ('pName', 'chairperson'), ('object', 'http://www.wikidata.org/entity/Q1740252')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('object', 'http://www.wikidata.org/entity/Q1860')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('object', 'http://www.wikidata.org/entity/Q19861084')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('object', 'http://www.wikidata.org/entity/Q334')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('object', 'http://www.wikidata.org/entity/Q408')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('object', 'http://www.wikidata.org/entity/Q5275956')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('object', 'http://www.wikidata.org/entity/Q99413897')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has par

Final query for this task

In [ ]:
# write your final query